In [1]:
import numpy as np
import pandas as pd
import nltk
import re

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
anime = pd.read_csv("/kaggle/input/anime-dataset/anime-dataset.csv")

In [3]:
anime.shape
anime.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [4]:
# rows_to_delete = anime[anime['Genres'] == 'Hentai'].index
anime = anime[~anime['Genres'].isin(['Boys Love',"Hentai",'Erotica','GirlsLove','UNKNOWN'])]
anime.shape

(18775, 24)

In [5]:
anime = anime[['anime_id','Name','Genres','Synopsis','Image URL']]

In [6]:
anime['Genres']=anime['Genres'].apply(lambda x : x.split(","))
anime['Genres']=anime['Genres'].apply(lambda x:[i.replace(" ","") for i in x])
anime['Genres']


0                [Action, AwardWinning, Sci-Fi]
1                              [Action, Sci-Fi]
2                   [Action, Adventure, Sci-Fi]
3        [Action, Drama, Mystery, Supernatural]
4            [Adventure, Fantasy, Supernatural]
                          ...                  
24898                              [AvantGarde]
24899                              [AvantGarde]
24900            [Comedy, Fantasy, SliceofLife]
24901              [Action, Adventure, Fantasy]
24902      [Action, Adventure, Fantasy, Sci-Fi]
Name: Genres, Length: 18775, dtype: object

In [7]:
anime['tags'] = anime['Genres'].astype(str) + " " + anime['Synopsis'].astype(str)
anime['tags'][0]


"['Action', 'AwardWinning', 'Sci-Fi'] Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his 

In [8]:
anime = anime[['anime_id', 'Name', 'tags', 'Image URL']]

In [9]:
anime['tags'] = anime['tags'].apply(lambda x: x.lower())
anime['tags']

0        ['action', 'awardwinning', 'sci-fi'] crime is ...
1        ['action', 'sci-fi'] another day, another boun...
2        ['action', 'adventure', 'sci-fi'] vash the sta...
3        ['action', 'drama', 'mystery', 'supernatural']...
4        ['adventure', 'fantasy', 'supernatural'] it is...
                               ...                        
24898    ['avantgarde'] music video for the song thaila...
24899    ['avantgarde'] music video for the song energy...
24900    ['comedy', 'fantasy', 'sliceoflife'] no descri...
24901    ['action', 'adventure', 'fantasy'] no descript...
24902    ['action', 'adventure', 'fantasy', 'sci-fi'] n...
Name: tags, Length: 18775, dtype: object

In [10]:
def clean_text(text):
   return re.sub(r'[^\w\s]', '', text)

# Apply the cleaning function to the 'tags' column
anime['tags'] = anime['tags'].apply(clean_text)

# Print the cleaned 'tags' column
print(anime['tags'])

0        action awardwinning scifi crime is timeless by...
1        action scifi another day another bountysuch is...
2        action adventure scifi vash the stampede is th...
3        action drama mystery supernatural robin sena i...
4        adventure fantasy supernatural it is the dark ...
                               ...                        
24898    avantgarde music video for the song thailand b...
24899    avantgarde music video for the song energy by ...
24900    comedy fantasy sliceoflife no description avai...
24901    action adventure fantasy no description availa...
24902    action adventure fantasy scifi no description ...
Name: tags, Length: 18775, dtype: object


In [11]:
ps = PorterStemmer()

def stem(text):
    words = word_tokenize(text)
    stemmed_words = [ps.stem(word) for word in words]
    return " ".join(stemmed_words)



In [12]:
anime['tags']=anime['tags'].apply(stem)
anime['tags']

0        action awardwin scifi crime is timeless by the...
1        action scifi anoth day anoth bountysuch is the...
2        action adventur scifi vash the stamped is the ...
3        action drama mysteri supernatur robin sena is ...
4        adventur fantasi supernatur it is the dark cen...
                               ...                        
24898    avantgard music video for the song thailand by...
24899    avantgard music video for the song energi by s...
24900    comedi fantasi sliceoflif no descript avail fo...
24901    action adventur fantasi no descript avail for ...
24902    action adventur fantasi scifi no descript avai...
Name: tags, Length: 18775, dtype: object

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000,stop_words="english")

cv_vectors = cv.fit_transform(anime['tags']).toarray()

In [14]:

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3000, stop_words="english")

# Fit and transform the 'tags' column
tfidf_vectors = tfidf_vectorizer.fit_transform(anime['tags']).toarray()




In [15]:
tfidf_vectors[0].shape

(3000,)

In [16]:
feature_names = tfidf_vectorizer.get_feature_names_out()
print(feature_names[0:10])


['10' '100' '10th' '11' '12' '13' '13th' '14' '14yearold' '15']


In [17]:
# similarity = cosine_similarity(vectors)
similarity_tf = cosine_similarity(tfidf_vectors, tfidf_vectors)
similarity_cv = cosine_similarity(cv_vectors, cv_vectors)
print(similarity_tf[0])


[1.         0.17155441 0.1269599  ... 0.         0.01665201 0.0371366 ]


In [18]:
similarity_tf.shape
similarity_cv.shape

(18775, 18775)

In [20]:
top_n = 5

# Get the indices of the top 100 similarity scores for each anime
top_5_indices_tf = np.argsort(-similarity_tf, axis=1)[:, :top_n]
top_5_indices_cv = np.argsort(-similarity_cv, axis=1)[:, :top_n]


# Create a list to store tuples of indices and values for each anime
similarity_list_tf = []
similarity_list_cv = []


# Iterate through each anime and store indices and values in the list
for i in range(similarity_tf.shape[0]):
    indices = top_5_indices_tf[i]
    values = similarity_tf[i, indices]
    similarity_list_tf.append(list(zip(indices, values)))
    
for i in range(similarity_cv.shape[0]):
    indices = top_5_indices_cv[i]
    values = similarity_cv[i, indices]
    similarity_list_cv.append(list(zip(indices, values)))



In [21]:
similarity_list_tf[0]

[(0, 1.0),
 (796, 0.2254945532436533),
 (2233, 0.21094643144910769),
 (12182, 0.21082226266250711),
 (1344, 0.2106557383692698)]

In [23]:
similarity_list_cv[0]

[(0, 0.9999999999999998),
 (796, 0.47665633097823756),
 (12182, 0.4680486795941482),
 (2233, 0.46581233568409464),
 (16833, 0.46124846117772816)]

In [ ]:
def recommend_list(anime_name):
    # Find the index of the given anime name in new_anime
    anime_index = anime[anime['Name'] == anime_name].index[0]
    
    # Extract the indices and values of the top 100 similarity scores for the given anime
    top_100_similarity_info_tf = similarity_list_tf[anime_index][1:]
    top_100_similarity_info_cv = similarity_list_cv[anime_index][1:]


    # Sort the top similarity scores in descending order
    sorted_similarity_info_tf = sorted(top_100_similarity_info_tf, key=lambda x: x[1], reverse=True)
    sorted_similarity_info_cv = sorted(top_100_similarity_info_cv, key=lambda x: x[1], reverse=True)


    # Print the recommended anime names
    print('Using TF')
    for anime_info in sorted_similarity_info_tf:
        index = anime_info[0]
        anime_name = anime.iloc[index]['Name']
        print(anime_name)
    print('Using CV')
    for anime_info in sorted_similarity_info_cv:
        index = anime_info[0]
        anime_name = anime.iloc[index]['Name']
        print(anime_name)


In [ ]:
recomended=recommend_list("Cowboy Bebop")

print(recomended)


#COWBOY BEBOP
# Black Lagoon
# Mushoku Tensei: Isekai Ittara Honki Dasu
# Saikyou Onmyouji no Isekai Tenseiki
# Black Lagoon: The Second Barrage
# Cike Wu Liuqi

#Monster
#['Jetter Mars', 'Araiguma Rascal', 'Kyoufu Densetsu Kaiki! Frankenstein', 'Cike Wu Liuqi', 'Shijou Saikyou no Daimaou, Murabito A ni Tensei suru']

#SPYXFAMILY
#['Shinobi no Ittoki', 'Blue Period', 'Baraou no Souretsu', 'Saikyou Onmyouji no Isekai Tenseiki', 'Cike Wu Liuqi']

#Overlord 
# Overlord II
# Overlord III
# Maou-sama, Retry!
# Overlord IV
# Sword Art Online Movie: Ordinal Scale
# Persona 3 the Movie 1: Spring of Birth
# Isekai Quartet 2
# Kusoge tte Iuna! Animation
# Death March kara Hajimaru Isekai Kyousoukyoku
# .hack//Intermezzo

In [24]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 73.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00


In [25]:
%%writefile app.py
import streamlit as st
import pandas as pd


def recommend(anime_name):
    anime_index = anime[anime['Name'] == anime_name].index[0]
    top_5_anime_list = similarity_list_cv[anime_index][:5]
    
    # Returning dummy recommendations for simplicity
    recommended_animes = [anime.iloc[i]['Name'] for i in range(len(animes))]
    recommended_animes_posters = [anime.iloc[i]['Image URL'] for i in range(len(animes))]

    return recommended_animes, recommended_animes_posters

st.title("Anime Recommender")

selected_movie_name = st.selectbox("Choose Anime", animes['Name'].values)

if st.button("Recommend"):
    names, posters = recommend(selected_movie_name)
    for name, poster in zip(names, posters):
        st.text(name)
        st.image(poster)

Writing app.py


In [26]:
!streamlit run app.py --server.port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://104.198.3.32:8501

^C
  Stopping...
